In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
from static_grader import grader

# PW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to use basic Python data structures, define functions, and control program flow. We will be using these concepts to perform some fundamental data wrangling tasks such as joining data sets together, splitting data into groups, and aggregating data into summary statistics.
**Please do not use `pandas` or `numpy` to answer these questions.**

We will be working with medical data from the British NHS on prescription drugs. Since this is real data, it contains many ambiguities that we will need to confront in our analysis. This is commonplace in data science, and is one of the lessons you will learn in this miniproject.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
%%bash
mkdir pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/201701scripts_sample.json.gz -nc -P ./pw-data
wget http://dataincubator-wqu.s3.amazonaws.com/pwdata/practices.json.gz -nc -P ./pw-data

## Loading the data

The first step of the project is to read in the data. We will discuss reading and writing various kinds of files later in the course, but the code below should get you started.

In [ ]:
import gzip
#import simplejson as json
import json

In [ ]:
with gzip.open('./pw-data/201701scripts_sample.json.gz', 'rb') as f:
    scripts = json.load(f)

with gzip.open('./pw-data/practices.json.gz', 'rb') as f:
    practices = json.load(f)

This data set comes from Britain's National Health Service. The `scripts` variable is a list of prescriptions issued by NHS doctors. Each prescription is represented by a dictionary with various data fields: `'practice'`, `'bnf_code'`, `'bnf_name'`, `'quantity'`, `'items'`, `'nic'`, and `'act_cost'`. 

In [ ]:
scripts[:2]

In [ ]:
len(scripts)

A [glossary of terms](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10686/Download-glossary-of-terms-for-GP-prescribing---presentation-level/pdf/PLP_Presentation_Level_Glossary_April_2015.pdf/) and [FAQ](http://webarchive.nationalarchives.gov.uk/20180328130852tf_/http://content.digital.nhs.uk/media/10048/FAQs-Practice-Level-Prescribingpdf/pdf/PLP_FAQs_April_2015.pdf/) is available from the NHS regarding the data. Below we supply a data dictionary briefly describing what these fields mean.

| Data field |Description|
|:----------:|-----------|
|`'practice'`|Code designating the medical practice issuing the prescription|
|`'bnf_code'`|British National Formulary drug code|
|`'bnf_name'`|British National Formulary drug name|
|`'quantity'`|Number of capsules/quantity of liquid/grams of powder prescribed|
| `'items'`  |Number of refills (e.g. if `'quantity'` is 30 capsules, 3 `'items'` means 3 bottles of 30 capsules)|
|  `'nic'`   |Net ingredient cost|
|`'act_cost'`|Total cost including containers, fees, and discounts|

The `practices` variable is a list of member medical practices of the NHS. Each practice is represented by a dictionary containing identifying information for the medical practice. Most of the data fields are self-explanatory. Notice the values in the `'code'` field of `practices` match the values in the `'practice'` field of `scripts`.

In [ ]:
practices[:2]

In the following questions we will ask you to explore this data set. You may need to combine pieces of the data set together in order to answer some questions. Not every element of the data set will be used in answering the questions.

## Question 1: summary_statistics

Our beneficiary data (`scripts`) contains quantitative data on the number of items dispensed (`'items'`), the total quantity of item dispensed (`'quantity'`), the net cost of the ingredients (`'nic'`), and the actual cost to the patient (`'act_cost'`). Whenever working with a new data set, it can be useful to calculate summary statistics to develop a feeling for the volume and character of the data. This makes it easier to spot trends and significant features during further stages of analysis.

Calculate the sum, mean, standard deviation, and quartile statistics for each of these quantities. Format your results for each quantity as a list: `[sum, mean, standard deviation, 1st quartile, median, 3rd quartile]`. We'll create a `tuple` with these lists for each quantity as a final result.

In [ ]:
def get(qty):
    cont = []
    for dict in scripts:
        cont.append(dict[qty])
        
    return cont
    
print(len(get('items')))
get('items')


In [ ]:
'''def describe(key):
n = len(scripts)
total = sum([script[key] for script in scripts])
avg = total / n
s = (sum([(script[key] - avg)**2 for script in scripts])/n)**(1/2)

sorted_values = sorted(scripts, key = lambda i: i[key])

q25 = sorted_values[int((1/4)*(n+1)-1)][key]
med = sorted_values[int((2/4)*(n+1)-1)][key]
q75 = sorted_values[int((3/4)*(n+1)-1)][key]

return (total, avg, s, q25, med, q75)'''



keys = scripts
def describe(key):
    
    n = len(scripts)
    total = sum(get(key))
    avg = sum(get(key))/len(get(key))
    s = (sum([(ky - avg)**2 for ky in get(key)])/(len(get(key)) - 1))**0.5
    
    
    sorted_values = sorted(scripts, key = lambda i: i[key])

    q25 = sorted_values[int((1/4)*(n+1)-1)][key]
    med = sorted_values[int((2/4)*(n+1)-1)][key]
    q75 = sorted_values[int((3/4)*(n+1)-1)][key]

    return (total, avg, s, q25, med, q75)

In [ ]:
summary = [('items', describe('items')),
           ('quantity', describe('quantity')),
           ('nic', describe('nic')),
           ('act_cost', describe('act_cost'))]

In [ ]:
print(summary)

In [ ]:
grader.score.pw__summary_statistics(summary)

## Question 2: most_common_item

Often we are not interested only in how the data is distributed in our entire data set, but within particular groups -- for example, how many items of each drug (i.e. `'bnf_name'`) were prescribed? Calculate the total items prescribed for each `'bnf_name'`. What is the most commonly prescribed `'bnf_name'` in our data?

To calculate this, we first need to split our data set into groups corresponding with the different values of `'bnf_name'`. Then we can sum the number of items dispensed within in each group. Finally we can find the largest sum.

We'll use `'bnf_name'` to construct our groups. You should have *5619* unique values for `'bnf_name'`.

We want to construct "groups" identified by `'bnf_name'`, where each group is a collection of prescriptions (i.e. dictionaries from `scripts`). We'll construct a dictionary called `groups`, using `bnf_names` as the keys. We'll represent a group with a `list`, since we can easily append new members to the group. To split our `scripts` into groups by `'bnf_name'`, we should iterate over `scripts`, appending prescription dictionaries to each group as we encounter them.

In [49]:
bnf_names = {d['bnf_name'] for d in scripts}
groups = {name: [] for name in bnf_names}
# Now loop through scripts and append the elements to the right key in groups dict
for script in scripts:
    groups[script['bnf_name']].append(script)

In [50]:
max_item =[max([(sum([member['items'] for member in group]),name) for name, group in groups.items()])[::-1]]

In [ ]:
def group_into_dictn (qty1, qty2):
    groupz = {Name: [] for Name in bnf_names}
    for script in scripts:
        if script[qty1] in groupz:
            groupz[script[qty1]].append(script[qty2])
    
    return groupz

In [ ]:
groups = group_into_dictn('bnf_name', 'items')
print(groups)

#groups = {name: [] for name in bnf_names}
#for script in scripts:
    # INSERT ...

In [ ]:
group_sum = []
for i in groups.values():
    group_sum.append(sum(i))
    
print(group_sum)

In [ ]:
print('Note that max sum = %d' %max(group_sum))
summed_group = dict(zip(bnf_names, group_sum))
print('Total qty of drugs sold i.e sum(group_sum) = %d' %sum(group_sum))
print('Confirmation of total qty of drugs sold i.e sum(get(\'items\')) = %d' %sum(get('items')))
print('length of sumed_group = %d' %len(summed_group))
summed_group

Now that we've constructed our groups we should sum up `'items'` in each group and find the `'bnf_name'` with the largest sum. The result, `max_item`, should have the form `[(bnf_name, item total)]`, e.g. `[('Foobar', 2000)]`.

In [ ]:
for key in summed_group:
    if summed_group[key] == max(group_sum):
        print(key)

In [ ]:
max_item = [("Omeprazole_Cap E/C 20mg", 113826)]
#max_item = [("", 0)]

**TIP:** If you are getting an error from the grader below, please make sure your answer conforms to the correct format of `[(bnf_name, item total)]`.

In [52]:
grader.score.pw__most_common_item(max_item)

Your score:  1.0


**Challenge:** Write a function that constructs groups as we did above. The function should accept a list of dictionaries (e.g. `scripts` or `practices`) and a tuple of fields to `groupby` (e.g. `('bnf_name')` or `('bnf_name', 'post_code')`) and returns a dictionary of groups. The following questions will require you to aggregate data in groups, so this could be a useful function for the rest of the miniproject.

In [53]:
def group_by_field(data, fields):
    names = {tuple(item[field] for field in fields) for item in data}
    groups = {name: [] for name in names}
    for item in data:
        name = tuple(item[field] for field in fields)
        groups[name].append(item)
    return groups

In [ ]:
'''def group_by_field(data, field1, field2 ):
    groups = {Name: [] for Name in bnf_names}
    for script in data:
        if script[field1] in groups:
            groups[script[field1]].append(script[field2])
    
    return groups'''

In [55]:
groups = group_by_field(scripts, ('bnf_name', 'items'))
test_max_item = 

assert test_max_item == max_item

AssertionError: 

## Question 3: postal_totals

Our data set is broken up among different files. This is typical for tabular data to reduce redundancy. Each table typically contains data about a particular type of event, processes, or physical object. Data on prescriptions and medical practices are in separate files in our case. If we want to find the total items prescribed in each postal code, we will have to _join_ our prescription data (`scripts`) to our clinic data (`practices`).

Find the total items prescribed in each postal code, representing the results as a list of tuples `(post code, total items prescribed)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes. Only include post codes if there is at least one prescription from a practice in that post code.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code.

We can join `scripts` and `practices` based on the fact that `'practice'` in `scripts` matches `'code'` in `practices`. However, we must first deal with the repeated values of `'code'` in `practices`. We want the alphabetically first postal codes.

In [ ]:
# format given by instructor
'''practice_postal = {}
for practice in practices:
    if practice['code'] in practice_postal:
        practice_postal[practice['code']] = ...
    else:
        practice_postal[practice['code']] = ...
    '''

In [ ]:
#My previous attempt. Was corect but didn't support exercise 4
'''def nget(qty):
    content = []
    for dict in practices:
        content.append(dict[qty])
        
    return content

def group_by_fieldd(data, field1, field2 ):
    unq_field1 = list(set(nget(field1)))
    ngroups = {Name: [] for Name in unq_field1}
    for script in data:
        if script[field1] in ngroups:
            ngroups[script[field1]].append(script[field2])
    
    return ngroups



practice_postal = group_by_fieldd(practices, 'code', 'post_code')
practice_postal
'''

In [ ]:
#Thi approach also worked fine
'''
practice_postal_sorted = []
for i in practice_postal.values():
    i.sort()
    practice_postal_sorted.append(i)
    
print(practice_postal_sorted)

practice_postal_sorted_01 = []
for lizt in practice_postal_sorted:
    practice_postal_sorted_01.append(lizt[0])

practice_postal_sorted_01
'''

In [ ]:
#Part of my working attempt. But this cell takes time to run
'''
for value in practice_postal_sorted_01:
    i = 0
    for key in practice_postal:
        practice_postal[key] = practice_postal_sorted_01[i]
        i += 1

practice_postal
'''

In [56]:
def group_by_field(data, fields):
    names = {tuple(item[field] for field in fields) for item in data}
    groups = {name: [] for name in names}
    for item in data:
        name = tuple(item[field] for field in fields)
        groups[name].append(item)
    return groups

practice_postal = {}
for practice in practices:
    code = practice['code']
    if code not in practice_postal:
        practice_postal[code] = practice['post_code']
    else:
        practice_postal[code] = min(practice_postal[code], practice['post_code'])

In [57]:
practice_postal['N81013']

'SK11 6JL'

**Challenge:** This is an aggregation of the practice data grouped by practice codes. Write an alternative implementation of the above cell using the `group_by_field` function you defined previously.

In [58]:
assert practice_postal['K82019'] == 'HP21 8TR'

Now we can join `practice_postal` to `scripts`.

In [ ]:
joined = scripts[:]

for script in joined:
    script['post_code'] = practice_postal[script['practice']]

pc_grouped = list(group_by_field(joined, ('post_code',)).items())

items_by_post = [(pc[0], sum(group['items'] for group in groups)) for pc, groups in pc_grouped]
items_by_post=sorted(items_by_post)
postal_totals = items_by_post[:100]

In [ ]:
#This code took even more time to run.
'''
for code in practice_postal:
    for dictn in joined:
        if dictn['practice'] == code:
            dictn['post_code'] = practice_postal[code]
        else:
            pass
joined
'''

In [ ]:
#Sorry code!
'''def get_from(qty, datab):
    content = []
    for dict in datab:
        content.append(dict[qty])
        
    return content
    
get_from('practice', joined)'''

In [ ]:
'''get_from('practice', scripts)[-1]'''

In [ ]:
'''
def nget(qty):
    content = []
    for dict in practices:
        content.append(dict[qty])
        
    return content

def get_from(qty, datab):
    content = []
    for dict in datab:
        content.append(dict[qty])
        
    return content
    


def group_by_field_from(data, key_field, field2 ):
    unq_key_field = list(set(get_from(key_field, data)))
    ngroups = {Name: [] for Name in unq_key_field}
    for script in data:
        if script[key_field] in ngroups:
            ngroups[script[key_field]].append(script[field2])
    
    return ngroups

group_by_field_from(joined, 'post_code', 'items')
'''

Finally we'll group the prescription dictionaries in `joined` by `'post_code'` and sum up the items prescribed in each group, as we did in the previous question.

In [ ]:
'''items_by_postt = group_by_field_from(joined, 'post_code', 'items')
items_by_post = dict(sorted(items_by_postt.items()))
assert items_by_post == items_by_postt'''

In [ ]:
'''postal_total = []
for k in items_by_post.keys():
    for v in items_by_post.values():
        if items_by_post[k] == v:
            postal_total.append((k, sum(v)))
        else:
            pass
postal_total'''

In [ ]:
#len(postal_total[:100])

In [59]:
joined = scripts[:]

for script in joined:
    script['post_code'] = practice_postal[script['practice']]

pc_grouped = list(group_by_field(joined, ('post_code',)).items())

items_by_post = [(pc[0], sum(group['items'] for group in groups)) for pc, groups in pc_grouped]
items_by_post=sorted(items_by_post)
postal_totals = items_by_post[:100]

In [62]:
#postal_totals = postal_total[:100]
#postal_totals = [('B11 4BW', 20673)] * 100

grader.score.pw__postal_totals(postal_totals)

Your score:  1.0


## Question 4: items_by_region

Now we'll combine the techniques we've developed to answer a more complex question. Find the most commonly dispensed item in each postal code, representing the results as a list of tuples (`post_code`, `bnf_name`, amount dispensed as proportion of total). Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** We'll continue to use the `joined` variable we created before, where we've chosen the alphabetically first postal code for each practice. Additionally, some postal codes will have multiple `'bnf_name'` with the same number of items prescribed for the maximum. In this case, we'll take the alphabetically first `'bnf_name'`.

Now we need to calculate the total items of each `'bnf_name'` prescribed in each `'post_code'`. Use the techniques we developed in the previous questions to calculate these totals. You should have 141196 `('post_code', 'bnf_name')` groups.

In [ ]:
#assert len(total_items_by_bnf_post) == 141196

In [ ]:
'''def group_by_fields(data, fields):
    ''' Receives a list and a tuple of fields and returns a list of dicts grouped by fields'''
    from itertools import groupby
    from operator import itemgetter
    grouper = itemgetter(*fields)
    result = []
    for key, grp in groupby(sorted(data, key = grouper), grouper): 
        # We must be sure key is a tuple
        if not isinstance(key, tuple):
            key = (key,)
        temp_dict = dict(zip([*fields], [*key]))
        temp_dict["items"] = sum(item["items"] for item in grp)
        result.append(temp_dict)
    return result


groups = group_by_fields(practices, ('post_code','bnf_name'))

total_items_by_bnf_post = groups'''

In [ ]:
#assert len(total_items_by_post) == 118

Let's use `total_items` to find the maximum item total for each postal code. To do this, we will want to regroup `total_items_by_bnf_post` by `'post_code'` only, not by `('post_code', 'bnf_name')`. First let's turn `total_items` into a list of dictionaries (similar to `scripts` or `practices`) and then group it by `'post_code'`. You should have 118 groups in the resulting `total_items_by_post` after grouping `total_items` by `'post_code'`.

In [ ]:
'''total_items = []
for pc, v in total_items_by_bnf_post.items():
    total_items.append({"post_code": pc[0], "bnf_name": pc[1], "items": v})

total_items_by_post = group_by_field(total_items, ('post_code',))'''

Now we will aggregate the groups in `total_by_item_post` to create `max_item_by_post`. Some `'bnf_name'` have the same item total within a given postal code. Therefore, if more than one `'bnf_name'` has the maximum item total in a given postal code, we'll take the alphabetically first `'bnf_name'`. We can do this by [sorting](https://docs.python.org/2.7/howto/sorting.html) each group according to the item total and `'bnf_name'`.

In [ ]:
#max_item_by_post = max(group_by_field(total_items, ('post_code',))) # use your logic here

In order to express the item totals as a proportion of the total amount of items prescribed across all `'bnf_name'` in a postal code, we'll need to use the total items prescribed that we previously calculated as `items_by_post`. Calculate the proportions for the most common `'bnf_names'` for each postal code. Format your answer as a list of tuples: `[(post_code, bnf_name, total)]`

In [ ]:
'''items_by_region = []

for pc, v in total_items_by_bnf_post.items():
    if max_item_by_post[pc[0]] == v: # only append on max items of that bnf_names
        items_by_region.append(pc + (v/total_items_by_post[pc[0]],))

items_by_region = sorted(items_by_region, key=lambda x: x[0])[:100]'''

In [ ]:
#items_by_region = [('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.0341508247)] * 100

In [ ]:
def group_by_field(data, fields):
    names = {tuple(item[field] for field in fields) for item in data}
    groups = {name: [] for name in names}
    for item in data:
        name = tuple(item[field] for field in fields)
        groups[name].append(item)
    return groups

In [63]:
practice_postal = {}
for practice in practices:
    code = practice['code']
    if code not in practice_postal:
        practice_postal[code] = practice['post_code']
    else:
        practice_postal[code] = min(practice_postal[code], practice['post_code'])

In [64]:
name_grouped = group_by_field(joined, ('post_code','bnf_name')).items()

total_items_by_bnf_post= {name: sum(group['items'] for group in groups) for name, groups in name_grouped}

total_items= [{'post_code': tup[0], 'bnf_name': tup[1], 'total' : tot} for tup, tot in list(total_items_by_bnf_post.items())]

total_items_by_post = group_by_field(total_items, ('post_code',))

max_items_by_post = [sorted(group, key=lambda mem: (-mem['total'], mem['bnf_name']))[0] for group in total_items_by_post.values()]

result= [(item['post_code'], item['bnf_name'],item['total'] /dict(items_by_post)[item['post_code']]) for item in max_items_by_post]

# Take the first 100 elements and pass it to the grader
items_by_region = sorted(result)[:100]

In [ ]:
#Past office hours youtube video link

https://www.youtube.com/playlist?list=PLeDYvCW3J3jnT66OZ8bFWrD6fK_Hey92i

In [65]:
grader.score.pw__items_by_region(items_by_region)

Your score:  1.0


*Copyright &copy; 2017 The Data Incubator.  All rights reserved.*